In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html
from dash import dash_table
from dash.dependencies import Input, Output

import base64

# Plotting / data tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Import your CRUD Python module and class
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Use the aacuser account and known password
username = "aacuser"
password = "JessieRose2021!"

# Instantiate CRUD class
db = AnimalShelter(username=username, password=password)

# Retrieve all documents from MongoDB
docs = db.read({})
df = pd.DataFrame.from_records(docs)

# Drop _id to avoid ObjectId issues in Dash DataTable
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# --- Logo setup ---
image_filename = "Grazioso Salvare Logo.png"   # file in the same folder as this notebook
encoded_image = base64.b64encode(open(image_filename, "rb").read())

# Radio filter options (rescue types)
filter_options = [
    {"label": "Reset (All Dogs)", "value": "reset"},
    {"label": "Water Rescue", "value": "water"},
    {"label": "Mountain or Wilderness Rescue", "value": "mountain"},
    {"label": "Disaster or Individual Tracking", "value": "disaster"},
]

app.layout = html.Div([
    # Logo and title row
    html.Div(
        children=[
            html.Img(
                src="data:image/png;base64,{}".format(encoded_image.decode()),
                style={"height": "150px", "padding": "10px"}
            ),
            html.Center(html.B(html.H1("Grazioso Salvare Rescue Dashboard"))),
            html.Center(html.H3("Dashboard by Morgun Leonard")),
        ]
    ),

    html.Hr(),

    # Interactive filter controls
    html.Div(
        children=[
            html.H4("Select Rescue Filter"),
            dcc.RadioItems(
                id="filter-type",
                options=filter_options,
                value="reset",
                labelStyle={"display": "inline-block", "margin-right": "20px"},
            ),
        ],
        style={"padding": "10px"}
    ),

    html.Hr(),

    # Interactive data table
    dash_table.DataTable(
        id="datatable-id",
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True}
            for i in df.columns
        ],
        data=df.to_dict("records"),

        # User-friendly features
        page_size=10,
        page_current=0,
        page_action="native",

        sort_action="native",
        sort_mode="multi",

        filter_action="native",

        row_selectable="single",
        selected_rows=[0],  # select first row by default so map has a point

        style_table={"overflowX": "auto"},
        style_cell={
            "textAlign": "left",
            "minWidth": "120px",
            "width": "120px",
            "maxWidth": "220px",
            "whiteSpace": "normal",
        },
        style_header={
            "backgroundColor": "lightgrey",
            "fontWeight": "bold"
        },
    ),

    html.Br(),
    html.Hr(),

    # Charts row: left = outcome chart, right = geolocation map
    html.Div(
        className="row",
        style={"display": "flex"},
        children=[
            html.Div(
                id="graph-id",
                className="col s12 m6",
            ),
            html.Div(
                id="map-id",
                className="col s12 m6",
            )
        ]
    )
])

#############################################
# Helper: build MongoDB query for each filter
#############################################

def build_filter_query(filter_type: str) -> dict:
    """
    Build a MongoDB query based on the selected rescue filter.
    Uses Rescue Type and Preferred Dog Breeds guidelines.
    """
    base = {"animal_type": "Dog"}

    # age thresholds in weeks (approx)
    three_years = 3 * 52  # 156
    four_years = 4 * 52   # 208
    six_years = 6 * 52    # 312

    # Example breed groups (aligned to course spec)
    water_breeds = [
        "Labrador Retriever Mix",
        "Chesapeake Bay Retriever Mix",
        "Newfoundland"
    ]

    mountain_breeds = [
        "German Shepherd",
        "Alaskan Malamute",
        "Old English Sheepdog",
        "Siberian Husky",
        "Rottweiler"
    ]

    disaster_breeds = [
        "German Shepherd",
        "Doberman Pinscher",
        "Rottweiler"
    ]

    tracking_breeds = [
        "Bloodhound",
        "Basset Hound",
        "Beagle",
        "Coonhound"
    ]

    # RESET – no filter
    if filter_type == "reset" or filter_type is None:
        return {}

    # WATER RESCUE
    if filter_type == "water":
        return {
            "$and": [
                base,
                {"age_upon_outcome_in_weeks": {"$lte": three_years}},
                {"breed": {"$in": water_breeds}}
            ]
        }

    # MOUNTAIN / WILDERNESS RESCUE
    if filter_type == "mountain":
        return {
            "$and": [
                base,
                {"age_upon_outcome_in_weeks": {"$lte": four_years}},
                {"breed": {"$in": mountain_breeds}}
            ]
        }

    # DISASTER OR INDIVIDUAL TRACKING
    if filter_type == "disaster":
        return {
            "$or": [
                {   # Disaster rescue
                    "$and": [
                        base,
                        {"age_upon_outcome_in_weeks": {"$lte": four_years}},
                        {"breed": {"$in": disaster_breeds}}
                    ]
                },
                {   # Individual tracking
                    "$and": [
                        base,
                        {"age_upon_outcome_in_weeks": {"$lte": six_years}},
                        {"breed": {"$in": tracking_breeds}}
                    ]
                }
            ]
        }

    # Fallback – no additional filter
    return {}

#############################################
# Interaction Between Components / Controller
#############################################

# 1) Filter data table via Mongo queries
@app.callback(
    Output("datatable-id", "data"),
    [Input("filter-type", "value")]
)
def update_dashboard(filter_type):
    """
    Update the DataTable data based on the rescue filter.
    Uses MongoDB via the CRUD module for each filter selection.
    """
    query = build_filter_query(filter_type)

    # Read from MongoDB with the appropriate filter
    records = db.read(query)

    if not records:
        return []

    filtered_df = pd.DataFrame.from_records(records)
    if "_id" in filtered_df.columns:
        filtered_df.drop(columns=["_id"], inplace=True)

    return filtered_df.to_dict("records")


# 2) Outcome-type chart driven by DataTable rows
@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "data")]
)
def update_graphs(table_data):
    """
    Build a chart based on the current rows in the DataTable.
    The data prop is always populated and updates when filters change.
    """
    if not table_data:
        return [html.Div("No data available for chart.")]

    dff = pd.DataFrame(table_data)

    if "outcome_type" not in dff.columns:
        return [html.Div("Column 'outcome_type' not found in data.")]

    fig = px.pie(
        dff,
        names="outcome_type",
        title="Outcome Type Distribution for Selected Filter",
    )

    return [dcc.Graph(figure=fig)]


# 3) Highlight selected column(s) in DataTable
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []

    return [{
        "if": {"column_id": i},
        "background_color": "#D2F3FF"
    } for i in selected_columns]


# 4) Geolocation map driven by DataTable selection
@app.callback(
    Output("map-id", "children"),
    [
        Input("datatable-id", "derived_virtual_data"),
        Input("datatable-id", "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, index):
    """
    Update Leaflet map based on the selected row in the DataTable.
    """
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    # Default to first row if none selected
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Safety guard for out-of-range selection
    if row >= len(dff):
        row = 0

    # Austin TX approx center
    center_lat = 30.75
    center_long = -97.48

    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[center_lat, center_long],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row]["location_lat"], dff.iloc[row]["location_long"]],
                    children=[
                        dl.Tooltip(dff.iloc[row]["breed"]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(dff.iloc[row].get("name", "Unknown")))
                        ])
                    ]
                )
            ]
        )
    ]


# Run app and display in JupyterLab/Codio; use port 8051 for your external URL
app.run_server(host="0.0.0.0", port=8051, debug=False)


 * Running on all addresses.
 * Running on http://10.56.248.10:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2025 21:22:50] "GET /_alive_9b68b291-e4c2-4c45-8ef1-68803eda59b3 HTTP/1.1" 200 -


Dash app running on http://0.0.0.0:8051/
